In [3]:
from openai import OpenAI
import openai
from dotenv import load_dotenv
import random
import os
import time
import pandas as pd

In [4]:
load_dotenv()

True

In [47]:
# client = OpenAI(
#   api_key="OPEN-API-KEY"
# ) #Replace OPEN-API-KEY with your own API Key

In [5]:

# Load prospect data
def load_prospects(filename):
    return pd.read_csv(filename)

# Simulate fetching customer company profile
def generate_company_profile(company_url):
    prompt = f"Analyze the website {company_url} and create a brief company profile suitable for sales outreach for promotion of products and services."
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        store=True,
        messages=[
            {"role": "system", "content": "You are is a marketing professional."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4
    )
    print(f"Generate company profile completion msg: {response.choices[0].message}")
    return response.choices[0].message.content.strip()

# Generate initial sales email
def generate_initial_email(prospect, company_profile):
    prompt = (
        f"The sender's company profile is: {company_profile}.\n"
        f"Write a cold outreach email to {prospect["Name"]} at {prospect["Company"]}.\n"
        f"Their business description is: {prospect["Description"]}.\n"
        f"Their email id is {prospect["Email"]}.\n"
        f"Write a persuasive sales email explaining how his company can benefit from the Sender's company in terms of promotion of his product or service.\n"
        f"The Email will be send by the sender to the prospect"
        f"The Email will be directly sent the sender's company sales representative"
    )
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        store=True,
        messages=[{"role": "system", "content": "You are is a sales copywriting expert."},
                  {"role": "user", "content": prompt}],
        temperature=0.4
    )
    print(f"Generate initial email completion msg: {response.choices[0].message}")
    return response.choices[0].message.content.strip()

# Simulate prospect replies
def simulate_prospect_reply():
    replies = [
        None,  # No reply
        "Thanks, but we're not looking for new tools right now.",
        "Sounds interesting. Can you send more details?",
        "Let's meet next Thursday at 2 PM.",
        "Please contact our CTO at cto@example.com."
    ]
    return random.choice(replies)

# Generate follow-up email based on simulated reply
def generate_follow_up_email(initial_email,reply, company_profile,prospect):
    if reply is None:
        return None
    prompt = (
        f"Initial Email sent was '{initial_email}'.\n"
        f"A prospect replied with: '{reply}'.\n"
        f"Write a professional follow-up email that acknowledges their message and moves the conversation forward.\n"
        f"Their email id is {prospect["Email"]}.\n"
        f"The sender's company profile is: {company_profile}.\n"
        f"The Email will we sent by Sender as a reply Email to the prompt"
        f"The Email will be directly sent the sender's company sales representative"
    )
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": "You are is a professional B2B sales representative."},
                  {"role": "user", "content": prompt}],
        temperature=0.4
    )
    print(f"Generate followup email completion msg: {response.choices[0].message}")
    return response.choices[0].message.content.strip()



# Main function
def process_prospects(input_csv, output_csv, customer_url):
    df = load_prospects(input_csv)
    company_profile = generate_company_profile(customer_url)
    output_data = []

    for _, prospect in df.iterrows():
        initial_email = generate_initial_email(prospect, company_profile)
        reply = simulate_prospect_reply()
        follow_up_email = generate_follow_up_email(initial_email,reply, company_profile,prospect) 
        follow_up_status = reply if follow_up_email else "No reply"
        output_data.append({
        "name": prospect["Name"],
        "company": prospect["Company"],
        "email": prospect["Email"],
        "initial_email": initial_email,
        "follow_up_status": follow_up_status,
        "follow_up_email": follow_up_email 
            })
        

        # To avoid hitting rate limits
        time.sleep(1)

    output_df = pd.DataFrame(output_data)
    output_df.to_csv(output_csv, index=False)
    print(f"Output written to {output_csv}")

# Example usage
if __name__ == "__main__":
    process_prospects("/Users/drago/Documents/Practicefiles/Data_files/prospects.csv", "output_emails.csv", "https://www.boseapac.com/en_in/about_bose.html") #Taking bose as an example of customer company URL 


Generate company profile completion msg: ChatCompletionMessage(content='**Company Profile: Bose Corporation**\n\n**Website:** [Bose APAC](https://www.boseapac.com/en_in/about_bose.html)\n\n**Overview:**\nBose Corporation is a globally recognized leader in audio technology, renowned for its innovative sound solutions that enhance the listening experience. Founded in 1964 by Dr. Amar G. Bose, the company has built a reputation for quality, performance, and customer satisfaction. With a commitment to research and development, Bose continually pushes the boundaries of audio technology, offering a diverse range of products that cater to both consumer and professional markets.\n\n**Key Offerings:**\n- **Consumer Audio Products:** Bose’s product lineup includes headphones, earbuds, soundbars, and home speaker systems, all designed to deliver superior sound quality and user experience. Their noise-cancelling headphones are particularly popular for their advanced technology and comfort.\n- **Pr